In [18]:
### Library Imports
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
import itertools
import warnings
warnings.filterwarnings("ignore")

In [19]:
### Function to Import Rate Data
def import_fx_data(tickers, start_date):
    data = pd.DataFrame()
    if isinstance(tickers, str):
        tickers = [tickers]
        
    for ticker in tickers:
        data[ticker] = yf.download(ticker, start = start_date)['Adj Close']
        
    # Reset index to make headings in the same row
    data.reset_index(inplace = True)
    # Convert values in date col to dt
    data['Date'] = pd.to_datetime(data['Date'])

    return data

### Call Function to Load in Rate Data
all_forex_pairs = [
    'EURUSD=X', 'GBPUSD=X', 'USDJPY=X', 'USDCHF=X', 'AUDUSD=X', 'NZDUSD=X',
    'USDCAD=X', 'EURGBP=X', 'EURJPY=X', 'EURCHF=X', 'EURAUD=X', 'EURNZD=X',
    'EURCAD=X', 'GBPAUD=X', 'GBPJPY=X', 'GBPCHF=X', 'AUDJPY=X', 'AUDCHF=X',
    'AUDNZD=X', 'NZDJPY=X', 'NZDCHF=X', 'CADJPY=X', 'CADCHF=X', 'CHFJPY=X'
]
start_date = '2021-01-01'
forex_data = import_fx_data(all_forex_pairs, start_date)
forex_data.tail()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Date,EURUSD=X,GBPUSD=X,USDJPY=X,USDCHF=X,AUDUSD=X,NZDUSD=X,USDCAD=X,EURGBP=X,EURJPY=X,...,GBPJPY=X,GBPCHF=X,AUDJPY=X,AUDCHF=X,AUDNZD=X,NZDJPY=X,NZDCHF=X,CADJPY=X,CADCHF=X,CHFJPY=X
897,2024-06-11,1.076426,1.272637,157.039993,0.89658,0.660895,0.612831,1.37612,0.84574,169.020004,...,199.843994,1.14096,103.751060,0.591900,1.078220,96.223000,0.54935,114.080002,0.65147,175.119995
898,2024-06-12,1.074033,1.274048,157.104996,0.89778,0.660770,0.614602,1.37549,0.84290,168.720993,...,200.143005,1.14372,103.805000,0.593226,1.075130,96.542999,0.55166,114.214996,0.65264,174.970001
899,2024-06-13,1.081315,1.279623,156.800003,0.89426,0.666290,0.618682,1.37219,0.84500,169.520004,...,200.630997,1.14400,104.468002,0.595810,1.076930,96.992996,0.55316,114.273003,0.65166,175.321335
900,2024-06-14,1.073883,1.275510,157.128006,0.89393,0.663310,0.616101,1.37410,0.84182,168.725998,...,200.419998,1.14024,104.214996,0.592500,1.076576,96.791000,0.55065,114.331001,0.65041,175.755432
901,2024-06-17,1.070205,1.267797,157.487000,0.89075,0.660310,0.612820,1.37380,0.84393,168.501007,...,199.645004,1.12920,NaN,NaN,1.077500,96.486000,0.54573,114.605003,0.64819,176.781998


In [20]:
### Compute Direct Hedge Daily Returns
def direct_daily_rets(fx_data):
    # Ensure 'Date' is set as the index for proper computation
    fx_data.set_index('Date', inplace=True)
    
    # Compute the daily returns of the forex data
    rets = fx_data.pct_change().dropna()
    
    # Reset index to make date a column again
    rets.reset_index(inplace=True)

    return rets

### Call Function to Calculate Daily Returns
daily_returns = direct_daily_rets(forex_data)
print(daily_returns.head())

        Date  EURUSD=X  GBPUSD=X  USDJPY=X  USDCHF=X  AUDUSD=X  NZDUSD=X  \
0 2021-01-04  0.005782  0.000818  0.000126 -0.016702  0.001211  0.002282   
1 2021-01-05  0.000074 -0.008228 -0.001104 -0.002106 -0.005355 -0.002792   
2 2021-01-06  0.003973  0.004270 -0.004335 -0.003472  0.012384  0.010951   
3 2021-01-07  0.003320 -0.000313  0.003370 -0.000125  0.006567  0.006736   
4 2021-01-08 -0.005645 -0.004084  0.007435  0.008063 -0.006141 -0.005514   

   USDCAD=X  EURGBP=X  EURJPY=X  ...  GBPJPY=X  GBPCHF=X  AUDJPY=X  AUDCHF=X  \
0  0.030138  0.011583  0.000174  ...  0.000829 -0.014968  0.015680 -0.007623   
1  0.004630  0.008300 -0.000909  ... -0.009373 -0.010321 -0.006695 -0.008367   
2 -0.008317 -0.000211 -0.000491  ... -0.000029  0.000811  0.008157  0.009320   
3 -0.000734  0.003513  0.006659  ...  0.002937 -0.000460  0.009922  0.006615   
4  0.001453 -0.001568  0.001746  ...  0.003406  0.003963  0.001553  0.002319   

   AUDNZD=X  NZDJPY=X  NZDCHF=X  CADJPY=X  CADCHF=X  CHFJPY=X 

In [21]:
### Create list of all possible unique pairs
def get_pairs(pairs):
    # Generate all unique combinations of pairs
    unique_pairings = list(itertools.combinations(pairs, 2))
    # Convert each tuple into a list
    unique_pairings = [list(pair) for pair in unique_pairings]
    
    return unique_pairings

### Function Call
pair_combos_list = get_pairs(all_forex_pairs)
pair_combos_list

[['EURUSD=X', 'GBPUSD=X'],
 ['EURUSD=X', 'USDJPY=X'],
 ['EURUSD=X', 'USDCHF=X'],
 ['EURUSD=X', 'AUDUSD=X'],
 ['EURUSD=X', 'NZDUSD=X'],
 ['EURUSD=X', 'USDCAD=X'],
 ['EURUSD=X', 'EURGBP=X'],
 ['EURUSD=X', 'EURJPY=X'],
 ['EURUSD=X', 'EURCHF=X'],
 ['EURUSD=X', 'EURAUD=X'],
 ['EURUSD=X', 'EURNZD=X'],
 ['EURUSD=X', 'EURCAD=X'],
 ['EURUSD=X', 'GBPAUD=X'],
 ['EURUSD=X', 'GBPJPY=X'],
 ['EURUSD=X', 'GBPCHF=X'],
 ['EURUSD=X', 'AUDJPY=X'],
 ['EURUSD=X', 'AUDCHF=X'],
 ['EURUSD=X', 'AUDNZD=X'],
 ['EURUSD=X', 'NZDJPY=X'],
 ['EURUSD=X', 'NZDCHF=X'],
 ['EURUSD=X', 'CADJPY=X'],
 ['EURUSD=X', 'CADCHF=X'],
 ['EURUSD=X', 'CHFJPY=X'],
 ['GBPUSD=X', 'USDJPY=X'],
 ['GBPUSD=X', 'USDCHF=X'],
 ['GBPUSD=X', 'AUDUSD=X'],
 ['GBPUSD=X', 'NZDUSD=X'],
 ['GBPUSD=X', 'USDCAD=X'],
 ['GBPUSD=X', 'EURGBP=X'],
 ['GBPUSD=X', 'EURJPY=X'],
 ['GBPUSD=X', 'EURCHF=X'],
 ['GBPUSD=X', 'EURAUD=X'],
 ['GBPUSD=X', 'EURNZD=X'],
 ['GBPUSD=X', 'EURCAD=X'],
 ['GBPUSD=X', 'GBPAUD=X'],
 ['GBPUSD=X', 'GBPJPY=X'],
 ['GBPUSD=X', 'GBPCHF=X'],
 

In [22]:
### Compute Hedge Ratio
def compute_hedge_ratio(pairs, returns):
    # Run linear regression to compute the hedge ratio
    rets1 = returns[pairs[0]]
    rets2 = returns[pairs[1]]
    # Add a constant to the independent variable
    rets1 = sm.add_constant(rets1)
    # Fit the regression model
    model = sm.OLS(rets2, rets1).fit()
    # Extract the hedge ratio
    hedge_ratio = model.params[pairs[0]]

    return hedge_ratio

In [26]:
### Define and Calculate Hedged and Unhedged Returns
def calculate_hedged_unhedged(long_pos, pairs, returns, hedge_ratio):
    # Compute Short Position
    short_pos = -hedge_ratio * long_pos

    # Compute daily returns for the long and short positions
    long_rets = returns[pairs[0]]
    short_rets = -hedge_ratio * returns[pairs[1]]

    # Compute the returns of the hedged and unhedged positions
    hedged_rets = long_rets + short_rets
    unhedged_rets = long_rets

    # Compute cumulative returns for the hedged and unhedged positions
    hedged_cumulative_rets = (1 + hedged_rets).cumprod() * long_pos
    unhedged_cumulative_rets = (1 + unhedged_rets).cumprod() * long_pos

    return short_pos, hedged_cumulative_rets, unhedged_cumulative_rets
    
### Append Long Position, Short Position, Hedged returns, and Unhedged Returns Columns
# Create an empty DataFrame to store hedge ratios
hedge_ratios_df = pd.DataFrame(columns = ['Pair1', 'Pair2', 'Hedge Ratio', 'Long Position', 'Short Position', 'Hedged Returns', 'Unhedged Returns'])
long_pos = 1000
# Loop over pair combinations to compute hedge ratio and returns
for pairing in pair_combos_list:
    try:
        hedge_ratio = compute_hedge_ratio(pairing, daily_returns)
        short_pos, hedged_cumulative_rets, unhedged_cumulative_rets = calculate_hedged_unhedged(long_pos, pairing, daily_returns, hedge_ratio)
        new_row = pd.DataFrame({
            'Pair1': [pairing[0]], 
            'Pair2': [pairing[1]], 
            'Hedge Ratio': [hedge_ratio], 
            'Long Position': [long_pos], 
            'Short Position': [short_pos],
            'Hedged Returns': [hedged_cumulative_rets.iloc[-1]], 
            'Unhedged Returns': [unhedged_cumulative_rets.iloc[-1]]
        })
        hedge_ratios_df = pd.concat([hedge_ratios_df, new_row], ignore_index=True)
    except Exception as e:
        print(f"Error computing hedge ratio for {pairing}: {e}")

# Display the DataFrame
print(hedge_ratios_df.tail())

        Pair1     Pair2  Hedge Ratio Long Position  Short Position  \
271  NZDCHF=X  CADCHF=X     0.465866          1000     -465.865874   
272  NZDCHF=X  CHFJPY=X    -0.249270          1000      249.269635   
273  CADJPY=X  CADCHF=X     0.407085          1000     -407.084635   
274  CADJPY=X  CHFJPY=X     0.590302          1000     -590.302148   
275  CADCHF=X  CHFJPY=X    -0.269728          1000      269.728089   

     Hedged Returns  Unhedged Returns  
271      873.641754        846.618084  
272      941.757082        846.618084  
273     1464.093373       1423.275612  
274     1116.523026       1423.275612  
275     1046.683980        933.184616  
